In [ ]:
import os
import sys
import subprocess
import tempfile

# --- CONFIGURATION ---
# Replace 'target.pdb' with the path to your actual PDB file
PDB_INPUT_PATH = "proteins\4LPK_swiss_model.pdb" 
OUTPUT_DIR = "prepared_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def run_protein_prep(pdb_path):
    """Simplified Meeko preparation for Notebooks"""
    
    if not os.path.exists(pdb_path):
        print(f"❌ Error: File {pdb_path} not found.")
        return None

    output_base = os.path.join(OUTPUT_DIR, "prepared_protein")
    pdbqt_path = f"{output_base}.pdbqt"

    print(f"⚙️ Starting preparation for: {pdb_path}")

    # Using sys.executable to ensure we use the same kernel environment
    cmd = [
        "python", "mk_prepare_receptor.py",
        "-i", pdb_path,
        "-o", output_base,
        "-p",                      # Add polar hydrogens
        "--charge_model", "gasteiger",
        "--default_altloc", "A"
    ]

    try:
        # Run the command
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        
        if os.path.exists(pdbqt_path):
            print(f"✅ Success! Prepared file saved to: {pdbqt_path}")
            print(f"STDOUT: {result.stdout[:200]}...") # Print snippet of output
            return pdbqt_path
        else:
            print("❌ Error: Command finished but .pdbqt file was not created.")
            
    except subprocess.CalledProcessError as e:
        print(f"❌ Meeko Error:\n{e.stderr}")
    except Exception as e:
        print(f"❌ Unexpected Error: {str(e)}")
    
    return None

# --- EXECUTION ---
prepared_file = run_protein_prep(PDB_INPUT_PATH)

if prepared_file:
    # Optional: Display the first 10 lines of the result
    with open(prepared_file, 'r') as f:
        head = [next(f) for _ in range(10)]
        print("\n--- PDBQT Preview ---")
        print("".join(head))

In [10]:
import base64
import json
import os
from IPython.display import HTML, display

def view_docking(file_path: str):
    """
    Reads a PDB file from the given path and displays it in Jupyter.
    
    Visual Style:
    - Protein: Spectrum Cartoon
    - Docked Ligand (UNL/LIG): Thick Magenta Sticks
    - Other Heteroatoms: Thin Green Sticks
    """
    
    # 1. Check if file exists
    if not os.path.exists(file_path):
        return f"Error: File not found at {file_path}"
    
    # 2. Read the file content
    try:
        with open(file_path, 'r') as f:
            pdb_text = f.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

    # 3. Prepare Data for JS
    pdb_json = json.dumps(pdb_text)
    
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <script src="https://3Dmol.org/build/3Dmol-min.js"></script>
        <style>
            body {{ margin: 0; padding: 0; overflow: hidden; background: white; }}
            #viewer_container {{ width: 100%; height: 500px; position: relative; }}
            .legend {{
                position: absolute; bottom: 10px; right: 10px;
                background: rgba(255, 255, 255, 0.9); padding: 8px 12px;
                border-radius: 5px; font-family: sans-serif; font-size: 12px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.2); z-index: 10; pointer-events: none;
            }}
            .box {{ display: inline-block; width: 10px; height: 10px; margin-right: 5px; border-radius: 50%; }}
        </style>
    </head>
    <body>
        <div id="viewer_container"></div>
        
        <div class="legend">
            <div><span class="box" style="background: linear-gradient(90deg, blue, red);"></span>Protein</div>
            <div style="margin-top:4px"><span class="box" style="background: magenta;"></span>Docked Ligand</div>
        </div>

        <script>
            window.onload = function() {{
                let element = document.getElementById('viewer_container');
                let config = {{ backgroundColor: 'white' }};
                let viewer = $3Dmol.createViewer(element, config);
                
                let pdbData = {pdb_json};
                
                // Load the model
                viewer.addModel(pdbData, "pdb");
                
                /* --- STYLING LAYERS --- */
                
                // Layer 1: Base Protein Style (Cartoon)
                // We apply this to everything first.
                viewer.setStyle({{}}, {{cartoon: {{color: 'spectrum'}}}});

                // Layer 2: General Heteroatoms (Water, Ions, etc)
                // Overwrite HETATM lines with Green Sticks
                viewer.addStyle(
                    {{hetflag: true}}, 
                    {{stick: {{colorscheme: 'greenCarbon', radius: 0.15}}}}
                );

                // Layer 3: The Docked Ligand (The most important part)
                // We specifically target residue names common in docking (UNL, LIG, DRG, UNK)
                // We make these FAT (radius 0.4) and Magenta.
                viewer.addStyle(
                    {{resn: ["UNL", "LIG", "DRG", "UNK"]}}, 
                    {{stick: {{colorscheme: 'magentaCarbon', radius: 0.4}}}}
                );

                /* ---------------------- */
                
                viewer.zoomTo();
                viewer.render();
            }};
        </script>
    </body>
    </html>
    """
    
    b64 = base64.b64encode(html_content.encode()).decode()
    return HTML(f'<iframe src="data:text/html;base64,{b64}" width="100%" height="520" frameborder="0"></iframe>')

# --- USAGE ---
# Just call the function with your file path:
# view_docking("your_docked_structure.pdb")

In [ ]:
view_docking("proteins\\4LPK.pdb")